<a href="https://colab.research.google.com/github/swoo-nam/app/blob/master/%EB%82%A8%EC%83%81%EC%9A%B0_BIO_DL_FPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 50.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from torch import nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

In [ ]:
DATA_PATH = "/content/drive/MyDrive/project/data/"
SEED = 42
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# EDA 데이터 불러오기

In [ ]:
train_df = pd.read_csv(f'{DATA_PATH}train_EDA.csv')
test_df = pd.read_csv(f'{DATA_PATH}test.csv')
test_df

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15
...,...,...,...,...,...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,4.207,306.443,2,1,7,4.207,55.13
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,-0.608,335.398,5,0,1,-1.736,70.16
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,1.792,349.383,3,1,3,1.792,69.72
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0.790,341.132,3,2,2,0.423,69.64


In [ ]:
test_df.isnull().sum()

id                            0
SMILES                        0
AlogP                         1
Molecular_Weight              0
Num_H_Acceptors               0
Num_H_Donors                  0
Num_RotatableBonds            0
LogD                          0
Molecular_PolarSurfaceArea    0
dtype: int64

In [ ]:
# 결측치 채우기
train_df["AlogP"] = np.where(pd.isna(train_df["AlogP"]), train_df["LogD"], train_df["AlogP"])
test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])

In [ ]:
train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

(0, 0)

In [ ]:
fps = []
for i, smiles in enumerate(train_df["SMILES"]):
    mol = Chem.MolFromSmiles(smiles)
    arr = np.zeros((1,))

    # rdkit의 fingerprint 함수를 사용해서 분자의 특성 수치화
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2048)
    DataStructs.ConvertToNumpyArray(fp, arr)
    fps.append(arr)

train_df["fps"] = fps
train_df['mol'] = train_df['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
train_df.head()

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,fps,mol
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c548113fd80>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c548113fd10>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c547fdd30d0>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c547fdd3140>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c547fdd3300>


In [ ]:
train_df.columns

Index(['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
       'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea', 'fps', 'mol'],
      dtype='object')

In [ ]:
yh = train_df['HLM']
ym = train_df['MLM']
x = train_df['fps']

# Split

In [ ]:
xh_train, xh_valid, yh_train, yh_valid = train_test_split(x, yh, test_size=.2)
xh_train.shape, xh_valid.shape, yh_train.shape, yh_valid.shape

((2788,), (698,), (2788,), (698,))

In [ ]:
xm_train, xm_valid, ym_train, ym_valid = train_test_split(x, ym, test_size=.2)
xm_train.shape, xm_valid.shape, ym_train.shape, ym_valid.shape

((2788,), (698,), (2788,), (698,))

In [ ]:
xh_valid

497     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
49      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1094    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2785    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2010    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
1832    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2766    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3483    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
887     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1729    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: fps, Length: 698, dtype: object

In [ ]:
len(xh_valid.iloc[0])

2048

In [ ]:
yh_valid

497     12.810
49      38.970
1094    20.247
2785    89.922
2010     4.304
         ...  
1832    57.372
2766    91.790
3483     1.790
887     23.650
1729     2.846
Name: HLM, Length: 698, dtype: float64

In [ ]:
def obj2float(data):
    return [i.astype(np.float32) for i in data]

In [ ]:
xh_train = obj2float(xh_train)
xh_valid = obj2float(xh_valid)

xm_train = obj2float(xm_train)
xm_valid = obj2float(xm_valid)

In [ ]:
def float2tensor(data):
    return torch.tensor(data)

In [ ]:
xh_train = float2tensor(xh_train)
xh_valid = float2tensor(xh_valid)
yh_train = float2tensor(yh_train.values)
yh_valid = float2tensor(yh_valid.values)

xm_train = float2tensor(xm_train)
xm_valid = float2tensor(xm_valid)
ym_train = float2tensor(ym_train.values)
ym_valid = float2tensor(ym_valid.values)

In [ ]:
def tensor2dataloader(data_x, data_y):
    return DataLoader(TensorDataset(data_x, data_y), batch_size=32, shuffle=True)

In [ ]:
h_train_dl = tensor2dataloader(xh_train, yh_train)
h_valid_dl = tensor2dataloader(xh_valid, yh_valid)
m_train_dl = tensor2dataloader(xm_train, ym_train)
m_valid_dl = tensor2dataloader(xm_valid, ym_valid)

# 모델 만들기

In [ ]:
class ModelH(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.model(x)

model_h = ModelH()
model_h.to(device)

ModelH(
  (model): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
class ModelM(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.model(x)

model_m = ModelM()
model_m.to(device)

ModelM(
  (model): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=256, out_features=1, bias=True)
  )
)

# 하이퍼파라미터 정의

In [ ]:
LR = 1e-3
optimizer_h = optim.Adam(model_h.parameters(), lr=LR)
optimizer_m = optim.Adam(model_m.parameters(), lr=LR)
loss_fn = nn.MSELoss()

# 학습 함수화

In [ ]:
def h_training(train_dl, test_dl):
    epochs = 1000
    train_losses = []
    val_losses = []
    best_rmse = float('inf')  # 초기값을 무한대로 설정하여 첫 번째 모델을 항상 저장
    patience = 0

    # train_loop
    for epoch in tqdm(range(epochs)):
        model_h.train()
        epoch_train_loss = 0
        for i, batch in enumerate(train_dl):
            x, y = batch[0].to(device), batch[1].to(device)

            x = x.to(torch.float32)
            y = y.to(torch.float32)

            optimizer_h.zero_grad()

            pred = model_h(x)
            train_loss = loss_fn(pred, y.view(-1, 1))
            train_loss.backward()
            optimizer_h.step()

            epoch_train_loss += train_loss.item()

        # test_loop
        model_h.eval()
        epoch_val_loss = 0
        with torch.inference_mode():
            for i, batch in enumerate(test_dl):
                x, y = batch[0].to(device), batch[1].to(device)

                x = x.to(torch.float32)
                y = y.to(torch.float32)

                pred = model_h(x)
                val_loss = loss_fn(pred, y.view(-1, 1))
                epoch_val_loss += val_loss.item()

        epoch_train_loss /= len(train_dl)
        epoch_val_loss /= len(test_dl)
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)

        if epoch % 10 == 0:
            print(f"[Epoch {epoch}] Train Loss: {epoch_train_loss}  Validation Loss: {epoch_val_loss}")

        # 현재 에포크에서의 RMSE 계산
        rmse = np.sqrt(epoch_val_loss)

        # 현재 모델이 이전까지의 최적 모델보다 좋다면 저장
        patience += 1
        if rmse < best_rmse:
            best_rmse = rmse
            patience = 0
            torch.save(model_h.state_dict(), "h_model.pth")

        if patience == 100:
            break

    return model_h, train_losses, val_losses

In [ ]:
h_model, h_train_losses, h_val_losses = h_training(h_train_dl, h_valid_dl)

  0%|          | 1/1000 [00:03<57:40,  3.46s/it]

[Epoch 0] Train Loss: 3525.2053319757633  Validation Loss: 3270.840587269176


  1%|          | 11/1000 [00:06<05:56,  2.77it/s]

[Epoch 10] Train Loss: 92.66429116509177  Validation Loss: 1487.4227572354403


  2%|▏         | 21/1000 [00:10<05:17,  3.08it/s]

...

 10%|█         | 101/1000 [00:37<04:43,  3.17it/s]

[Epoch 100] Train Loss: 34.412929155609824  Validation Loss: 1583.7271340110085


 11%|█         | 111/1000 [00:40<04:44,  3.12it/s]

[Epoch 110] Train Loss: 26.131330425089057  Validation Loss: 1504.1987970525568


 11%|█         | 111/1000 [00:40<05:25,  2.73it/s]


In [ ]:
def m_training(train_dl, test_dl):
    epochs = 1000
    train_losses = []
    val_losses = []
    best_rmse = float('inf')  # 초기값을 무한대로 설정하여 첫 번째 모델을 항상 저장
    patience = 0

    # train_loop
    for epoch in tqdm(range(epochs)):
        model_m.train()
        epoch_train_loss = 0
        for i, batch in enumerate(train_dl):
            x, y = batch[0].to(device), batch[1].to(device)

            x = x.to(torch.float32)
            y = y.to(torch.float32)

            optimizer_m.zero_grad()

            pred = model_m(x)
            train_loss = loss_fn(pred, y.view(-1, 1))
            train_loss.backward()
            optimizer_m.step()

            epoch_train_loss += train_loss.item()

        # test_loop
        model_m.eval()
        epoch_val_loss = 0
        with torch.inference_mode():
            for i, batch in enumerate(test_dl):
                x, y = batch[0].to(device), batch[1].to(device)

                x = x.to(torch.float32)
                y = y.to(torch.float32)

                pred = model_m(x)
                val_loss = loss_fn(pred, y.view(-1, 1))
                epoch_val_loss += val_loss.item()

        epoch_train_loss /= len(train_dl)
        epoch_val_loss /= len(test_dl)
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)

        if epoch % 10 == 0:
            print(f"[Epoch {epoch}] Train Loss: {epoch_train_loss}  Validation Loss: {epoch_val_loss}")

        # 현재 에포크에서의 RMSE 계산
        rmse = np.sqrt(epoch_val_loss)

        # 현재 모델이 이전까지의 최적 모델보다 좋다면 저장
        patience += 1
        if rmse < best_rmse:
            best_rmse = rmse
            patience = 0
            torch.save(model_m.state_dict(), "m_model.pth")

        if patience == 100:
            break

    return model_m, train_losses, val_losses

In [ ]:
m_model, m_train_losses, m_val_losses = m_training(m_train_dl,m_valid_dl)

  0%|          | 1/1000 [00:00<05:54,  2.82it/s]

[Epoch 0] Train Loss: 2276.733720259233  Validation Loss: 2019.7351351651278


  1%|          | 11/1000 [00:04<06:36,  2.49it/s]

[Epoch 10] Train Loss: 95.66676963459362  Validation Loss: 1377.29248046875


  2%|▏         | 21/1000 [00:08<07:45,  2.10it/s]

...

 20%|██        | 201/1000 [01:09<05:52,  2.27it/s]

[Epoch 200] Train Loss: 15.696936358105052  Validation Loss: 1466.1424005681818


 21%|██        | 211/1000 [01:16<08:11,  1.60it/s]

[Epoch 210] Train Loss: 18.955537760799583  Validation Loss: 1489.4515380859375


 22%|██▏       | 217/1000 [01:20<04:49,  2.70it/s]


In [ ]:
sns.set_style('whitegrid')

# 테스트 데이터 적용

In [ ]:
test_df.head()

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15


In [ ]:
fps = []
for i, smiles in enumerate(test_df["SMILES"]):
    mol = Chem.MolFromSmiles(smiles)
    arr = np.zeros((1,))
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2048)
    DataStructs.ConvertToNumpyArray(fp, arr)

    fps.append(arr)

test_df["fps"] = fps
test_df['mol'] = test_df['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
test_df.head()

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,fps,mol
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c547f9d3300>
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c547f9d3680>
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c547f9d3060>
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c547f9d3610>
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",<rdkit.Chem.rdchem.Mol object at 0x7c547f9d2ff0>


In [ ]:
test_df.isnull().sum()

id                            0
SMILES                        0
AlogP                         0
Molecular_Weight              0
Num_H_Acceptors               0
Num_H_Donors                  0
Num_RotatableBonds            0
LogD                          0
Molecular_PolarSurfaceArea    0
fps                           0
mol                           0
dtype: int64

In [ ]:
y = test_df['fps']

In [ ]:
y = obj2float(y)
y = float2tensor(y)

# df 저장

In [ ]:
train_df.to_csv(f'{DATA_PATH}train_fps_mol.csv', index=False, encoding="utf-8-sig")
test_df.to_csv(f'{DATA_PATH}test_fps_mol.csv', index=False, encoding="utf-8-sig")

# 추론하기

In [ ]:
def evaluation(model, y):
    model.eval()
    predictions = []
    with torch.inference_mode():
        preds = model(y.to(device))
        preds = [pred[0] for pred in preds.cpu().detach().numpy().tolist()]
        predictions += preds

    return predictions

In [ ]:
hlm_pred = evaluation(h_model, y)
hlm_pred

[22.862279891967773,
 90.90746307373047,
 40.4796028137207,
 35.77260971069336,
 34.472042083740234,
 89.88595581054688,
 8.257468223571777,
 70.66459655761719,
 39.593379974365234,
 75.08429718017578,
 37.324466705322266,
 86.81605529785156,
 83.42481994628906,
 39.52527618408203,
 50.79903793334961,
 85.24664306640625,
 51.0267219543457,
 82.28064727783203,
 67.93508911132812,
 63.78059005737305,
 55.99406814575195,
 8.92685317993164,
 16.227420806884766,
 71.3164291381836,
 31.902711868286133,
 67.51829528808594,
 59.11632537841797,
 78.61697387695312,
 44.92783737182617,
 43.43198776245117,
 57.32974624633789,
 64.69776153564453,
 14.171419143676758,
 55.767757415771484,
 85.65479278564453,
 70.92852020263672,
 43.799713134765625,
 26.175979614257812,
 73.22061920166016,
 29.808948516845703,
 80.27592468261719,
 72.17632293701172,
 13.354008674621582,
 77.17755889892578,
 16.714675903320312,
 43.58517074584961,
 58.13057327270508,
 38.08539581298828,
 77.15707397460938,
 62.6884651

In [ ]:
mlm_pred = evaluation(m_model, y)
mlm_pred

[2.4758858680725098,
 75.92789459228516,
 22.754589080810547,
 18.422910690307617,
 2.8713765144348145,
 78.07654571533203,
 38.17283630371094,
 54.636566162109375,
 51.20612335205078,
 43.96848678588867,
 53.44221878051758,
 68.64505767822266,
 74.5276107788086,
 65.09821319580078,
 2.682894706726074,
 57.8735237121582,
 43.138370513916016,
 76.30921936035156,
 26.909936904907227,
 23.062644958496094,
 64.49142456054688,
 13.274322509765625,
 8.124702453613281,
 36.50593185424805,
 59.60358810424805,
 36.185604095458984,
 30.826839447021484,
 60.365379333496094,
 60.85665512084961,
 34.9194221496582,
 46.825923919677734,
 51.74966812133789,
 37.095829010009766,
 27.837461471557617,
 61.10691452026367,
 54.71098709106445,
 75.7323226928711,
 26.91462516784668,
 52.13564682006836,
 10.78996467590332,
 34.58319854736328,
 54.77495193481445,
 62.1619758605957,
 58.540470123291016,
 17.751331329345703,
 5.0993218421936035,
 57.371917724609375,
 20.176422119140625,
 69.27387237548828,
 83.4

#

In [ ]:
y = test_df['fps']

In [ ]:
y = obj2float(y)
y = float2tensor(y)

# 제출

In [ ]:
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submission

,id,MLM,HLM
0,TEST_000,0,0
1,TEST_001,0,0
2,TEST_002,0,0
3,TEST_003,0,0
4,TEST_004,0,0
...,...,...,...
478,TEST_478,0,0
479,TEST_479,0,0
480,TEST_480,0,0
481,TEST_481,0,0


In [ ]:
len(submission['MLM']) , len(mlm_pred) , len(hlm_pred)

(483, 483, 483)

In [ ]:
submission['MLM'] = mlm_pred
submission['HLM'] = hlm_pred

In [ ]:
submission

,id,MLM,HLM
0,TEST_000,7.115345,40.730370
1,TEST_001,76.155663,92.186554
2,TEST_002,53.729660,27.046124
3,TEST_003,24.218344,44.362000
4,TEST_004,44.385895,75.993088
...,...,...,...
478,TEST_478,51.954147,85.551117
479,TEST_479,80.050735,83.675377
480,TEST_480,22.785559,45.153053
481,TEST_481,24.447937,49.627983


In [ ]:
submission.to_csv(f"{DATA_PATH}dl_sub_1.csv", index=False)

In [ ]:
submission.to_csv("dl_sub_1.csv", index=False)

- 제출 점수 높았던 예측값과 앙상블

In [ ]:
ml = pd.read_csv(f"{DATA_PATH}ml_sub_1.csv")
dl_fp = pd.read_csv(f"{DATA_PATH}dl_sub_1.csv")

In [ ]:
submission['MLM'] = (ml['MLM'] + dl_fp['MLM']) / 2
submission['HLM'] = (ml['HLM'] + dl_fp['HLM']) / 2
submission

,id,MLM,HLM
0,TEST_000,18.249304,42.632796
1,TEST_001,65.110596,84.933916
2,TEST_002,43.043190,35.638452
3,TEST_003,40.321714,58.756131
4,TEST_004,55.908683,75.898244
...,...,...,...
478,TEST_478,31.352587,55.309582
479,TEST_479,76.811283,83.662790
480,TEST_480,32.468803,56.639146
481,TEST_481,43.416987,58.099265


In [ ]:
submission.to_csv(f"{DATA_PATH}ensemble_sub_1.csv", index=False)

In [ ]:
submission.to_csv("ensemble_sub_1.csv", index=False)